In [1]:
import pandas as pd
import numpy as np
from HFCs_hmimCl_PR import configuration
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest
from pyomo.opt import TerminationCondition, SolverStatus

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
m1 = ConcreteModel()

m1.fs = FlowsheetBlock(default={"dynamic": False})

m1.fs.properties = GenericParameterBlock(default=configuration)

m1.fs.F101 = Flash(default={"property_package": m1.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

In [3]:
data = pd.read_csv('R32_hmimCl_10_2.csv')
data2 = pd.read_csv('R125_hmimCl_10_2.csv')

In [4]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [5]:
# Make outlet arrays
IL_x = np.zeros(len(data["1"]))
IL_y = np.zeros(len(data["1"]))
R32_x = np.zeros(len(data["1"]))
R32_y = np.zeros(len(data["1"]))
R125_x = np.zeros(len(data["1"]))
R125_y = np.zeros(len(data["1"]))

alpha1 = np.zeros(len(data["1"]))

m1.fs.F101.inlet.flow_mol.fix(10)
m1.fs.F101.inlet.temperature.fix(298)
m1.fs.F101.inlet.mole_frac_comp[0, "hmimCl"].fix(0.5) 
m1.fs.F101.inlet.pressure.fix(100000)
m1.fs.F101.vap_outlet.pressure.fix(100000)
m1.fs.F101.vap_outlet.temperature.fix(298.15)

for i in range(len(data["0"])):
    
    m1.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.009)
    m1.fs.properties.PR_kappa_A['R125', 'R32'].fix(0.007)
    m1.fs.properties.PR_kappa_A['R32', 'hmimCl'].fix(data["1"].iloc[i])
    m1.fs.properties.PR_kappa_A['hmimCl', 'R32'].fix(data["0"].iloc[i])
    m1.fs.properties.PR_kappa_A['hmimCl', 'R125'].fix(data2["0"].iloc[i])
    m1.fs.properties.PR_kappa_A['R125', 'hmimCl'].fix(data2["1"].iloc[i])

    m1.fs.F101.inlet.flow_mol.fix(10)
    m1.fs.F101.inlet.temperature.fix(298)
    m1.fs.F101.inlet.mole_frac_comp[0, "hmimCl"].fix(0.7) 
    m1.fs.F101.inlet.mole_frac_comp[0, "R32"].fix(0.2) 
    m1.fs.F101.inlet.mole_frac_comp[0, "R125"].fix(0.1)

    m1.fs.F101.inlet.pressure.fix(100000)

    m1.fs.F101.vap_outlet.pressure.fix(100000)
    m1.fs.F101.vap_outlet.temperature.fix(298.15)

    try:
        m1.fs.F101.initialize(outlvl=idaeslog.CRITICAL)              

        # solve the model
        status = solver.solve(m1, tee = False)

        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):

            IL_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "hmimCl"])
            IL_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "hmimCl"])
            R32_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R32"]) 
            R32_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R32"])
            R125_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R125"])
            R125_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R125"])

            print(i)
            
            alpha1[i] = (R125_y[i]/R125_x[i])/(R32_y[i]/R32_x[i])

    except ValueError:
        alpha1[i] = 'NaN'
       
        
results = pd.DataFrame(alpha1)
results.to_csv('hmimCl_alpha_10_2.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [6]:
print(alpha1)

[0.55854789 0.64426695 0.63461361 0.67443803 0.65065234 0.76201901
 0.65153334 0.64212273 0.61269569 0.61852241 0.55894158 0.61025523
 0.59661418 0.56583282 0.79395248 0.70342793 0.63620999 0.6153908
 0.62003235 0.62334706 0.61291294 0.63591911 0.65595505 0.60138151
 0.60361607 0.60955905 0.65898314 0.56860594 0.57587087 0.77232452
 0.63117053 0.6709341  0.54871254 0.57417314 0.6452446  0.60968402
 0.66008686 0.66529394 0.64567356 0.60232159 0.69816969 0.60553927
 0.54572347 0.57745554 0.59330218 0.60402949 0.59731281 0.57794005
 0.63351843 0.60795701 0.6122573  0.60285462 0.61410695 0.74245115
 0.65243308 0.73223013 0.61295178 0.61028102 0.61417446 0.60177166
 0.56220427 0.6100873  0.60420907 0.60559811 0.75042994 0.60961238
 0.58823441 0.64222432 0.60613052 0.60879653 0.6181401  0.61307827
 0.61256496 0.7285973  0.60794886 0.51811094 0.7589995  0.69996382
 0.61005218 0.59320787 0.60309814 0.69022056 0.68128358 0.64263965
 0.71942738 0.60472144 0.60381446 0.70268832 0.60998937 0.61152